# Projekt 4 Super k-NN

Celem projektu jest stworzenie zupełnie nowego zespołowego klasyfikatora k-NN  i porównania jego jakości, czasów jego uczenia i odpowiedzi ze standardowym klasyfikatorem SVM. 

DoD.
Należy sporządzić raport z projektu.

1. Zbiór danych: TNG, ok 18000 próbek, 20 klas. Zbiór danych MNIST (70000 próbek 10 klas). Odnośnie TNG wykorzystujemy gotowe dane reprezentujące tekst blogów w postaci wektorów (dostarcza prowadzący). Dane dekorelujemy wykorzystując transformatę PCA.

2. Z jednego zbioru danych tworzymy kilka sub-zestawów danych (>=5 <=10) na różnych zestawach cech (maski mogą być losowane w sposób random, ale nie powinny być gęste). Maski mogą mieć różną długość. Prawdopodobieństwo wystąpienia cechy w zestawie może być proporcjonalne do jej istotności (np. mierzonej wielkością wartości własnych po transformacie PCA). Jednak nie może być takiej cechy, która nie dostała się do żadnego zestawu. 

3. Liczymy średnią przynależność każdej próbki do danej klasy na bazie klasyfikatora k-NN dla każdego sub-zestawu danych. Dokonujemy fuzji wyników klasyfikacji (jakiej?) dla każdej próbki po sub-zestawach danych. 

4. Jak zmieni się jakość klasyfikatora w zależności od k?

5. Wyniki jakości klasyfikatorów oceniać na bazie krzyżowej-walidacji, (accuracy - Loss-błąd, Krzywa ROC, Precision-Recall, (pola pod krzywymi) F1). 

## Utils

In [1]:
from sklearn.datasets import fetch_openml
import random
import numpy as np
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, f1_score, log_loss, plot_roc_curve, precision_recall_curve,
plot_precision_recall_curve, average_precision_score, hinge_loss, precision_score, recall_score, classification_report)
from sklearn.svm import SVC
from scipy import stats
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.ensemble import ExtraTreesClassifier

def divide_dataset(X: np.ndarray,
                   y: np.ndarray,
                   sub_num: int = 10,
                   rows_size: float = 0.6,
                   cols_size: float = 0.7):
    # get feature importances
    tree = ExtraTreesClassifier()
    tree.fit(X, y)
    feature_importances = tree.feature_importances_
    feature_importances /= feature_importances.sum()

    # prepare indices
    row_indices = list(range(X.shape[0]))
    col_indices = list(range(X.shape[1]))
    num_rows = round(X.shape[0] * rows_size)
    num_cols = round(X.shape[1] * cols_size)

    used_cols = set()
    subparts = []
    col_masks = []
    
    for i in range(sub_num):
        # randomly, uniformly sample rows
        rows = np.random.choice(row_indices,
                                size=num_rows,
                                replace=True)

        # randomly sample X columns with probability distribution relative to
        # the features' importances
        cols = np.random.choice(col_indices,
                                size=num_cols,
                                replace=False,
                                p=feature_importances)
        if i == sub_num - 1:
            # force usage of columns not used before
            used_cols |= set(cols)
            not_used_cols = set(col_indices) - used_cols
            not_used_cols = np.fromiter(not_used_cols,
                                        int,
                                        len(not_used_cols))
            cols = np.sort(np.concatenate((cols, not_used_cols)))

        
        X_part = X[rows, :]
        X_part = X_part[:, cols]
        
        y_part = y[rows]

        used_cols |= set(cols)
        subparts.append((X_part, y_part))
        col_masks.append(cols)
        
    return subparts, col_masks

In [3]:
class SuperKNN:
    def __init__(self, classes_count, n_neighbors=3, classifiers=[], masks=[],
                 current_predict_proba=[]):
        self.__classes_count = classes_count
        self.__n_neighbors = n_neighbors
        self.__classifiers = classifiers
        self.__masks = masks
        self.__current_predict_proba = current_predict_proba
    
    def fit(self, x_train, y_train, rows_size=0.8, sub_num=10):
        divided_dataset_with_masks = divide_dataset(x_train, y_train, sub_num=sub_num,rows_size=rows_size)
        self.__classifiers = []
        self.__fit_classifiers(divided_dataset_with_masks[0])  
        self.__masks = divided_dataset_with_masks[1]
        
    def __fit_classifiers(self, subset):
        for row in subset:
            x, y = row
            neigh = KNeighborsClassifier(n_neighbors=self.__n_neighbors)
            neigh.fit(x, y)
            self.__classifiers.append(neigh)

    def predict(self, x):
        if not self.__masks:
            raise Exception('You need to call fit first')
        self.__current_predictions = []
        
        for row in x:
            current_result = int(np.argmax(self.__get_votes(row)))
            self.__current_predictions.append(current_result)
        return self.__current_predictions
    
    def __get_votes(self, row):
        votes = [0] * self.__classes_count

        for i, c in enumerate(self.__classifiers):
            
            # match shape to train set shape
            temp_x = row[self.__masks[i]] 

            for prediction in c.predict([temp_x]):
                votes[int(prediction)] += 1

        return votes
    
    def __get_current_predict_proba(self, row):
        votes = self.__get_votes(row)
        return [x / self.__classes_count for x in votes]
    
    def predict_proba(self, x):
        self.__current_predict_proba = []
        
        for row in x:
            cur_pred_proba = self.__get_current_predict_proba(row)
            self.__current_predict_proba.append(cur_pred_proba)
        return self.__current_predict_proba
    
    def decision_function(self, x_test):
        return self.predict_proba(x_test)
    
    @property
    def classifiers(self):
        return self.__classifiers
    
    @classifiers.setter
    def classifiers(self, classifiers):
        self.__classifiers = classifiers
    
    def get_params(self, deep=True):
        return {"classes_count": self.__classes_count, "n_neighbors": self.__n_neighbors,
                "classifiers": self.__classifiers, "masks": self.__masks, 
                "current_predict_proba": self.__current_predict_proba }
        

In [4]:
class ReportCreator():
    def __init__(self):
        self.columns = ['name',
                        'time',
                        'accuracy (cross-val)',
                        'accuracy',
                        'f1-score',
                        'precision',
                        'recall',
                        'loss'
                        ]
        self.base_df = pd.DataFrame(columns = self.columns)
        
    def get_df_row(self, report):
        return pd.DataFrame(report, columns=self.columns, index=[0])
    
    def concat_new_row(self, row_report):
        self.base_df = pd.concat([self.base_df, self.get_df_row(row_report)], ignore_index=True)

    def create_partial_report(self, classifier, X_train, X_test, y_train, y_test, result, end, start, kernel=None, C=None, k=None, m=None):
        partial_report = classification_report(y_test, result, output_dict=True)['weighted avg']
        if kernel is None or C is None:
            if m is None:
                partial_report['name'] = 'Super k-NN, k={}'.format(k)
            else:
                partial_report['name'] = 'Super k-NN, m={}'.format(m)
        else:
            partial_report['name'] = 'SVM, C = {}, kernel: {}'.format(C, kernel)
        partial_report['time'] = f'{end - start} s'
        partial_report['accuracy'] = accuracy_score(y_true=y_test, y_pred=result)
        partial_report['f1-score'] = f1_score(y_true=y_test, y_pred=result, average='macro')
        partial_report['accuracy (cross-val)'] = np.mean(cross_val_score(classifier, X_train, y_train, cv=5, scoring='accuracy'))
        partial_report['loss'] = log_loss(y_test, classifier.decision_function(X_test))
    
        self.concat_new_row(partial_report)
    
    def get_report(self):
        return self.base_df.sort_values(by='accuracy', ascending=False)


def evaluate_svm(report_creator, X_train, X_test, y_train, y_test, kernel='linear', C=1):
    start = time.time()
    classifier =  SVC(kernel=kernel, C=C, probability=True)
    classifier.fit(X_train, y_train)
    result = classifier.predict(X_test)
    end = time.time()

    report_creator.create_partial_report(classifier, X_train, X_test, y_train, y_test, result, end, start, kernel, C)

def evaluate_all(X_train, X_test, y_train, y_test):
    report_creator = ReportCreator()

    for C in [1, 5]:
        for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
            evaluate_svm(report_creator, X_train, X_test, y_train, y_test, C=C, kernel=kernel)
    return report_creator.get_report()

# MNIST

In [5]:
mnist = fetch_openml("mnist_784", data_home="data/mnist_784", cache=True)

### Super k-NN

In [6]:
zipped_mnist = list(zip(mnist.data, mnist.target))
mnist_random = random.sample(zipped_mnist, 7000)
x, y = zip(*(mnist_random))
x = np.asarray(x, dtype=np.int)
y = np.asarray(y, dtype=np.int)

In [7]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x) 

pca = PCA(n_components=30)
x_pca = pca.fit_transform(x_scaled)
x_train, x_test, y_train, y_test = train_test_split(x_pca, y, train_size=0.2, test_size=0.02, random_state=1)

In [8]:
report_creator = ReportCreator()

for i in [1, 3, 5, 10, 20]:
    start = time.time()
    sknn = SuperKNN(10, i)
    sknn.fit(x_train, y_train, sub_num=10)
    result = sknn.predict(x_test)
    end = time.time()
    
    report_creator.create_partial_report(sknn, x_train, x_test, y_train, y_test, result, end, start, k=i)

report_creator.get_report()

,name,time,accuracy (cross-val),accuracy,f1-score,precision,recall,loss
0,"Super k-NN, k=1",1.1210641860961914 s,0.868571,0.864286,0.858960,0.883658,0.864286,2.195019
4,"Super k-NN, k=20",1.1350648403167725 s,0.843571,0.850000,0.854732,0.875715,0.850000,1.744003
2,"Super k-NN, k=5",1.1270644664764404 s,0.869286,0.842857,0.844824,0.856934,0.842857,1.968764
1,"Super k-NN, k=3",1.1160635948181152 s,0.879286,0.835714,0.833793,0.857824,0.835714,1.266884
3,"Super k-NN, k=10",1.1290645599365234 s,0.859286,0.835714,0.838093,0.850239,0.835714,2.423662


In [9]:
#rozna liczba klasyfikatorow prostych
report_creator = ReportCreator()

for i in [1, 2, 3, 4, 5]:
    start = time.time()
    sknn = SuperKNN(10, 3)
    sknn.fit(x_train, y_train, sub_num=i)
    result = sknn.predict(x_test)
    end = time.time()
    
    report_creator.create_partial_report(sknn, x_train, x_test, y_train, y_test, result, end, start, k=3, m=i)

report_creator.get_report()

,name,time,accuracy (cross-val),accuracy,f1-score,precision,recall,loss
0,"Super k-NN, m=1",0.33601927757263184 s,0.880714,0.814286,0.807233,0.848617,0.814286,5.986721
3,"Super k-NN, m=4",0.5830333232879639 s,0.872857,0.814286,0.813107,0.824256,0.814286,2.412170
2,"Super k-NN, m=3",0.49602842330932617 s,0.870714,0.807143,0.804996,0.832568,0.807143,3.516587
1,"Super k-NN, m=2",0.4110236167907715 s,0.873571,0.785714,0.774646,0.831329,0.785714,4.577905
4,"Super k-NN, m=5",0.6730387210845947 s,0.876429,0.785714,0.780171,0.813219,0.785714,2.441426


### SVM

In [10]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, train_size=0.008, test_size=0.002, random_state=1)

In [11]:
scaler_mnist = StandardScaler().fit(X_train)
x_train = scaler_mnist.transform(X_train)
x_test = scaler_mnist.transform(X_test)

In [12]:
evaluate_all(X_train, X_test, y_train, y_test)

,name,time,accuracy (cross-val),accuracy,f1-score,precision,recall,loss
6,"SVM, C = 5, kernel: rbf",1.5140864849090576 s,0.898214,0.864286,0.851268,0.874909,0.864286,2.173525
2,"SVM, C = 1, kernel: rbf",1.4140808582305908 s,0.885714,0.850000,0.839032,0.863135,0.850000,2.172164
0,"SVM, C = 1, kernel: linear",0.7680437564849854 s,0.844643,0.828571,0.810836,0.840569,0.828571,2.182134
4,"SVM, C = 5, kernel: linear",0.7360420227050781 s,0.844643,0.828571,0.810836,0.840569,0.828571,2.182134
5,"SVM, C = 5, kernel: poly",1.1290645599365234 s,0.835714,0.828571,0.822310,0.848176,0.828571,2.170233
3,"SVM, C = 1, kernel: sigmoid",0.9670553207397461 s,0.826786,0.785714,0.771453,0.810759,0.785714,2.173838
7,"SVM, C = 5, kernel: sigmoid",0.6250357627868652 s,0.812500,0.778571,0.749644,0.790616,0.778571,2.186161
1,"SVM, C = 1, kernel: poly",1.1380650997161865 s,0.823214,0.764286,0.754023,0.795958,0.764286,2.420810


# TNG

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(shuffle=True, subset='train', random_state=42, categories=categories, remove=('headers', 'footers', 'quotes'))
twenty_test = fetch_20newsgroups(shuffle=True, subset='test', random_state=42, categories=categories, remove=('headers', 'footers', 'quotes'))

In [14]:
x_train = twenty_train.data[:3000]
y_train = twenty_train.target[:3000]
x_test = twenty_test.data[:600]
y_test = twenty_test.target[:600]

# convert text to vectors
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

### Super k-NN

In [15]:
x_train = preprocessing.scale(x_train, with_mean=False)
x_test = preprocessing.scale(x_test, with_mean=False)

svd = TruncatedSVD(n_components=70)
x_train = svd.fit_transform(x_train)
x_test = svd.transform(x_test)

In [16]:
report_creator = ReportCreator()

for i in [1, 3, 5, 10, 20]:
    start = time.time()
    sknn = SuperKNN(4, i)
    sknn.fit(x_train, y_train)
    result = sknn.predict(x_test)
    end = time.time()
    
    report_creator.create_partial_report(sknn, x_train, x_test, y_train, y_test, result, end, start, k=i)

report_creator.get_report()

,name,time,accuracy (cross-val),accuracy,f1-score,precision,recall,loss
4,"Super k-NN, k=20",5.631321907043457 s,0.674338,0.636667,0.628765,0.680527,0.636667,5.405456
3,"Super k-NN, k=10",5.345305919647217 s,0.675670,0.628333,0.622532,0.669855,0.628333,4.835113
2,"Super k-NN, k=5",5.21729850769043 s,0.676559,0.626667,0.619168,0.661820,0.626667,3.910581
1,"Super k-NN, k=3",5.155294895172119 s,0.668138,0.618333,0.609777,0.654335,0.618333,3.555147
0,"Super k-NN, k=1",5.186296701431274 s,0.653948,0.616667,0.612678,0.641590,0.616667,4.221572


In [17]:
#rozna liczba klasyfikatorow prostych
report_creator = ReportCreator()

for i in [1, 2, 3, 4, 5]:
    start = time.time()
    sknn = SuperKNN(4, 3)
    sknn.fit(x_train, y_train, sub_num=i)
    result = sknn.predict(x_test)
    end = time.time()
    
    report_creator.create_partial_report(sknn, x_train, x_test, y_train, y_test, result, end, start, k=3, m=i)

report_creator.get_report()

,name,time,accuracy (cross-val),accuracy,f1-score,precision,recall,loss
4,"Super k-NN, m=5",2.730156183242798 s,0.660154,0.626667,0.623852,0.664646,0.626667,5.577311
3,"Super k-NN, m=4",2.287130832672119 s,0.668573,0.618333,0.613699,0.643446,0.618333,6.041411
0,"Super k-NN, m=1",0.9900565147399902 s,0.670790,0.585000,0.577235,0.610036,0.585000,13.758280
2,"Super k-NN, m=3",1.7811017036437988 s,0.660596,0.571667,0.571385,0.622461,0.571667,8.080876
1,"Super k-NN, m=2",1.3530774116516113 s,0.669448,0.538333,0.533970,0.620806,0.538333,10.290205


### SVM

In [18]:
evaluate_all(x_train, x_test, y_train, y_test)

,name,time,accuracy (cross-val),accuracy,f1-score,precision,recall,loss
0,"SVM, C = 1, kernel: linear",2.7751588821411133 s,0.755423,0.710000,0.702451,0.717408,0.710000,2.025043
4,"SVM, C = 5, kernel: linear",10.548603296279907 s,0.745231,0.693333,0.683335,0.703951,0.693333,2.142098
6,"SVM, C = 5, kernel: rbf",2.5741474628448486 s,0.570658,0.551667,0.519917,0.588048,0.551667,3.835705
7,"SVM, C = 5, kernel: sigmoid",2.389136552810669 s,0.527678,0.475000,0.433229,0.513672,0.475000,4.590906
2,"SVM, C = 1, kernel: rbf",2.9401681423187256 s,0.467867,0.435000,0.358028,0.406550,0.435000,5.451667
3,"SVM, C = 1, kernel: sigmoid",2.6741530895233154 s,0.439078,0.385000,0.312283,0.364199,0.385000,6.546081
1,"SVM, C = 1, kernel: poly",2.3571348190307617 s,0.270271,0.278333,0.108866,0.077469,0.278333,8.225863
5,"SVM, C = 5, kernel: poly",2.3631348609924316 s,0.272928,0.278333,0.108866,0.077469,0.278333,8.229599
